In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)


def GetGaussHermiteRecursive(x, n):
    
    if n==0:
        poly = 1
    elif n==1:
        poly = 2*x
    else:
        poly = 2*x*GetGaussHermiteRecursive(x, n-1) - 2*(n-1)*GetGaussHermiteRecursive(x, n-2)
    
    return poly
    
def GetDiffHermite(n, x):
    Pn = GetGaussHermiteRecursive(x, n)

    return sym.diff(Pn, x, 1)    
    

def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

def GetRootsGH(f,df,x,tolerancia = 14):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)
    
        croot = np.round( root, tolerancia )
        
        if croot not in Roots:
            Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

def GetAllRootsGH(n):

    xn = np.linspace(-(np.sqrt((4*n)-1)),(np.sqrt((4*n)-1),100))
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetGaussHermiteRecursive(i, x))
        DHermite.append(GetDiffHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRootsGH(poly,Dpoly,xn)
    
    return Roots

def GetWeightsHermite (n):
    
    Roots = GetAllRootsGH(n)

    

    DHermite = []
    
    for i in range(n+1):
        DHermite.append(GetDiffHermite(i,x))
    
    Dpoly = sym.lambdify([x],DHermite[n-1],'numpy')
    Weights = ((2**(n-1))*np.math.factorial(n)*np.sqrt(np.pi))/((n**2)*(Dpoly(Roots))**2) 
    
    return Weights  
  

    I = 0
    for i in range(len(Weights)):
        I += Weights[i]*f(roots[i])
        
    return I